In [1]:
# interactive figure
%matplotlib inline

In [4]:
# pip install pygmt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 kB 5.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
# auto reload modules
%load_ext autoreload
%autoreload 2

import warnings
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML, display

import pygmt

In [5]:
ds = sio.loadmat('/Users/jcai/Desktop/SWOT/Model/ZHAO23/ZHAO23-MD1-M2-SWOT19-SN4.mat', squeeze_me=True, struct_as_record=False);

omega = 2*np.pi/(12.42*3600)
t = np.arange(0,24.5,0.5)*3600

ssh = ds['ZHAO23'].NorthWestSUM.ssh[:,:,None]*np.cos(-omega*t[None,None,:] + ds['ZHAO23'].NorthWestSUM.phs[:,:,None])

In [30]:
fig = pygmt.Figure()
with fig.subplot(1,1, figsize=(6,4.5)):
    oahu = [-131, -120, 32, 41]
    fig.coast(resolution="f", shorelines="1p", region=oahu, projection="M5c")


    Ch = fig.grdcontour(ds['ZHAO23'].NorthWestSUM.lon, ds['ZHAO23'].NorthWestSUM.lat, ssh[:,:,0], 22, cmap='Spectral_r');
    cbar = fig.colorbar(Ch, fraction=0.05)
    cbar.set_label(r'Surface height anomaly', labelpad=-33, fontsize=10)
    fig.set_title(f't = {0*0.5} hours', y=0.999);

fig.show()

TypeError: grdcontour() takes 2 positional arguments but 5 were given

In [ ]:
oahu = [-128, -122, 32, 42]
pygmt.Figure().coast(resolution="f", shorelines="1p", region=oahu, projection="M5c")

Ch = ax.contourf(ds['ZHAO23'].NorthWestSUM.lon, ds['ZHAO23'].NorthWestSUM.lat, ssh[:,:,0], 22, cmap='Spectral_r');
cbar = fig.colorbar(Ch, ax=ax, fraction=0.05)
cbar.set_label(r'Surface height anomaly', labelpad=-33, fontsize=10)
ax.set_title(f't = {0*0.5} hours', y=0.999);


In [ ]:
def update(frame):
    global Ch
    # for each frame, get new data and clear old data stored on each artist
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        for coll in Ch.collections:
            coll.remove()
    # update the plot
    Ch = ax.contourf(ds['ZHAO23'].NorthWestSUM.lon, ds['ZHAO23'].NorthWestSUM.lat, ssh[:,:,frame], 22, cmap='Spectral_r')
    ax.set_title(f't = {frame*0.5} hours', y=0.999)
    return Ch

ani = animation.FuncAnimation(fig=fig, func=update, frames=np.size(t), 
                              interval=400, repeat_delay=800)# blit=True);#
plt.close()
display(HTML(ani.to_jshtml()));
# ani.save('./SouthSUM.mp4', writer='ffmpeg', fps=3, dpi=200);